# Standard Diagnostics for Simulated Investment Panels

This notebook expands the diagnostics into a **standard analysis pipeline** for the model output.

## What statistics matter most in this class of models?

### 1) Investment behavior
- **Mean/median investment rate (`I_total / K`)**: central measure of capital adjustment intensity.
- **Dispersion of investment rates** (std, IQR, p10/p90): captures heterogeneity and lumpy adjustment.
- **Inaction rate** (`|I_total|` near zero): key moment under fixed or non-convex adjustment costs.
- **Positive vs negative investment shares**: identifies asymmetry and irreversibility/frictions.
- **Semi-annual timing decomposition** (`I` vs `Delta_I`): measures how much adjustment happens initially vs after mid-year information.

### 2) State dependence
- **Investment by demand state** (`log_D`) and volatility state (`log_sigma`): tests policy function monotonicity and uncertainty effects.
- **Regime contrasts** (low vs high volatility quantiles): reduced-form uncertainty-investment relationship.

### 3) Dynamics
- **Persistence/autocorrelation of investment rates**: serial dependence and gradual adjustment.
- **Year profiles of average investment/profit/capital**: transition dynamics and stationarity checks.

### 4) Cross-sectional heterogeneity
- **Between-firm dispersion** (distribution of firm-level means): persistent heterogeneity across firms.
- **Within-firm volatility**: cyclical variation and sensitivity to shocks.

### 5) Economic outcomes
- **Profit levels and dispersion**.
- **Capital distribution moments**.
- **Correlation of investment with demand and volatility**.

The cells below implement all these diagnostics for one or multiple simulation outputs.


In [ ]:
%pip install -q pandas numpy matplotlib pyfixest


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")
pd.options.display.float_format = "{:.4f}".format

# Update this path if your simulation CSVs are elsewhere.
folder_path = "../output/simulations"
pattern = "panel_data*.csv"


## Load panels
The loader searches for `panel_data*.csv` and creates a dictionary `{scenario_name: DataFrame}`.


In [ ]:
def load_panels(folder_path, pattern="panel_data*.csv"):
    files = sorted(glob.glob(os.path.join(folder_path, pattern)))
    if not files:
        raise FileNotFoundError(
            f"No files matching '{pattern}' were found in {folder_path}. "
            "Run simulation first or update `folder_path`."
        )

    panels = {}
    for path in files:
        name = os.path.splitext(os.path.basename(path))[0]
        df = pd.read_csv(path)

        # Safety: reconstruct investment rate if absent
        if "I_rate" not in df.columns and {"I_total", "K"}.issubset(df.columns):
            df["I_rate"] = df["I_total"] / df["K"].replace(0, np.nan)

        panels[name] = df

    return panels

panels = load_panels(folder_path, pattern=pattern)
print(f"Loaded {len(panels)} scenario(s): {list(panels.keys())}")


## Core diagnostic statistics
This block computes a rich set of moments for each scenario.


In [ ]:
def autocorr_by_firm(df, var, lags=(1, 2)):
    out = {}
    grouped = df.sort_values(["firm_id", "year"]).groupby("firm_id")
    for lag in lags:
        vals = []
        for _, g in grouped:
            x = g[var]
            if len(x) > lag and x.std() > 0:
                vals.append(x.autocorr(lag=lag))
        out[f"{var}_ac{lag}_mean"] = np.nanmean(vals) if vals else np.nan
    return out


def scenario_diagnostics(df, near_zero=1e-6):
    stats = {}

    # Basic panel size
    stats["n_obs"] = len(df)
    stats["n_firms"] = df["firm_id"].nunique()
    stats["T"] = df.groupby("firm_id")["year"].max().median()

    # Investment moments
    i = df["I_rate"]
    stats["I_rate_mean"] = i.mean()
    stats["I_rate_median"] = i.median()
    stats["I_rate_std"] = i.std()
    stats["I_rate_p10"] = i.quantile(0.10)
    stats["I_rate_p90"] = i.quantile(0.90)
    stats["I_rate_iqr"] = i.quantile(0.75) - i.quantile(0.25)

    # Lumpy adjustment / inaction / sign asymmetry
    total_i = df["I_total"]
    stats["inaction_rate"] = (total_i.abs() <= near_zero).mean()
    stats["positive_invest_share"] = (total_i > near_zero).mean()
    stats["negative_invest_share"] = (total_i < -near_zero).mean()

    # Timing decomposition
    abs_i = df["I"].abs().sum()
    abs_di = df["Delta_I"].abs().sum()
    denom = abs_i + abs_di
    stats["share_initial_abs_adjustment"] = abs_i / denom if denom > 0 else np.nan
    stats["share_midyear_abs_adjustment"] = abs_di / denom if denom > 0 else np.nan

    # State dependence and reduced-form associations
    stats["corr_Irate_logD"] = df[["I_rate", "log_D"]].corr().iloc[0, 1]
    stats["corr_Irate_logSigma"] = df[["I_rate", "log_sigma"]].corr().iloc[0, 1]

    # High- vs low-volatility regime gap
    q25, q75 = df["log_sigma"].quantile([0.25, 0.75])
    low = df.loc[df["log_sigma"] <= q25, "I_rate"].mean()
    high = df.loc[df["log_sigma"] >= q75, "I_rate"].mean()
    stats["Irate_low_vol"] = low
    stats["Irate_high_vol"] = high
    stats["Irate_high_minus_low_vol"] = high - low

    # Capital and profit moments
    stats["K_mean"] = df["K"].mean()
    stats["K_std"] = df["K"].std()
    stats["profit_mean"] = df["profit"].mean()
    stats["profit_std"] = df["profit"].std()

    # Dynamics: autocorrelation (averaged over firms)
    stats.update(autocorr_by_firm(df, "I_rate", lags=(1, 2)))
    stats.update(autocorr_by_firm(df, "profit", lags=(1, 2)))

    # Heterogeneity: distribution of firm means and within-firm volatility
    firm_mean = df.groupby("firm_id")["I_rate"].mean()
    firm_std = df.groupby("firm_id")["I_rate"].std()
    stats["between_firm_std_of_mean_Irate"] = firm_mean.std()
    stats["mean_within_firm_std_Irate"] = firm_std.mean()

    return stats


def diagnostics_table(panels):
    rows = []
    for scenario, df in panels.items():
        row = {"scenario": scenario}
        row.update(scenario_diagnostics(df))
        rows.append(row)
    return pd.DataFrame(rows).set_index("scenario").sort_index()

summary = diagnostics_table(panels)
summary


## Pretty summary (selected headline moments)


In [ ]:
headline_cols = [
    "n_firms", "T", "I_rate_mean", "I_rate_std", "inaction_rate",
    "positive_invest_share", "negative_invest_share",
    "share_initial_abs_adjustment", "share_midyear_abs_adjustment",
    "Irate_high_minus_low_vol", "I_rate_ac1_mean", "profit_ac1_mean"
]

summary[headline_cols].sort_values("I_rate_mean", ascending=False)


## Visual diagnostics


In [ ]:
def plot_investment_rate_distribution(panels, bins=60):
    n = len(panels)
    fig, axes = plt.subplots(n, 1, figsize=(10, 3.2 * n), squeeze=False)
    for ax, (scenario, df) in zip(axes.flatten(), panels.items()):
        ax.hist(df["I_rate"].dropna(), bins=bins, alpha=0.85)
        ax.axvline(df["I_rate"].mean(), linestyle="--", linewidth=1.5, label="mean")
        ax.set_title(f"Investment rate distribution — {scenario}")
        ax.set_xlabel("I_rate")
        ax.legend()
    plt.tight_layout()


plot_investment_rate_distribution(panels)


In [ ]:
def plot_time_profiles(panels):
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))
    for scenario, df in panels.items():
        by_year = df.groupby("year").agg(
            I_rate_mean=("I_rate", "mean"),
            profit_mean=("profit", "mean"),
            K_mean=("K", "mean")
        )
        axes[0].plot(by_year.index, by_year["I_rate_mean"], label=scenario)
        axes[1].plot(by_year.index, by_year["profit_mean"], label=scenario)
        axes[2].plot(by_year.index, by_year["K_mean"], label=scenario)

    axes[0].set_title("Mean investment rate over time")
    axes[1].set_title("Mean profit over time")
    axes[2].set_title("Mean capital over time")
    for ax in axes:
        ax.set_xlabel("Year")
        ax.legend(fontsize=8)
    plt.tight_layout()


plot_time_profiles(panels)


In [ ]:
def binned_state_dependence(df, state_col, target_col="I_rate", n_bins=12):
    x = df[state_col]
    bins = pd.qcut(x, q=n_bins, duplicates="drop")
    out = df.groupby(bins, observed=False)[target_col].mean().reset_index()
    out["mid"] = out[state_col].apply(lambda iv: (iv.left + iv.right) / 2)
    return out[["mid", target_col]]


def plot_state_dependence(panels):
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))

    for scenario, df in panels.items():
        b1 = binned_state_dependence(df, "log_D", "I_rate")
        b2 = binned_state_dependence(df, "log_sigma", "I_rate")

        axes[0].plot(b1["mid"], b1["I_rate"], marker="o", label=scenario)
        axes[1].plot(b2["mid"], b2["I_rate"], marker="o", label=scenario)

    axes[0].set_title("Investment vs demand state (binned)")
    axes[0].set_xlabel("log_D bin midpoint")
    axes[0].set_ylabel("Mean I_rate")

    axes[1].set_title("Investment vs volatility state (binned)")
    axes[1].set_xlabel("log_sigma bin midpoint")
    axes[1].set_ylabel("Mean I_rate")

    for ax in axes:
        ax.legend(fontsize=8)
    plt.tight_layout()


plot_state_dependence(panels)


In [ ]:
def plot_adjustment_timing(panels):
    scenarios = list(panels.keys())
    initial = []
    midyear = []

    for _, df in panels.items():
        abs_i = df["I"].abs().sum()
        abs_di = df["Delta_I"].abs().sum()
        total = abs_i + abs_di
        initial.append(abs_i / total if total > 0 else np.nan)
        midyear.append(abs_di / total if total > 0 else np.nan)

    x = np.arange(len(scenarios))
    w = 0.37

    fig, ax = plt.subplots(figsize=(10, 4.5))
    ax.bar(x - w/2, initial, width=w, label="Initial I share")
    ax.bar(x + w/2, midyear, width=w, label="Mid-year ΔI share")

    ax.set_xticks(x)
    ax.set_xticklabels(scenarios, rotation=25, ha="right")
    ax.set_ylim(0, 1)
    ax.set_title("Absolute adjustment timing decomposition")
    ax.legend()
    plt.tight_layout()


plot_adjustment_timing(panels)


## Optional: keep simple pairwise scenario checks
These preserve your earlier quick tests but now fit into the expanded workflow.


In [ ]:
def quick_pair_plot(panels, s1, s2, firm_id=1):
    if s1 not in panels or s2 not in panels:
        print(f"Skipping: {s1} or {s2} not found.")
        return

    fig, ax = plt.subplots(figsize=(9, 4))
    panels[s1].query("firm_id == @firm_id").plot(x="year", y="I_total", ax=ax, label=f"{s1}")
    panels[s2].query("firm_id == @firm_id").plot(x="year", y="I_total", ax=ax, label=f"{s2}")
    ax.set_title(f"Firm {firm_id}: total investment comparison")
    plt.tight_layout()


# Edit names if needed based on files in your folder.
quick_pair_plot(panels, "panel_data_s11", "panel_data_s12", firm_id=1)
quick_pair_plot(panels, "panel_data_s21", "panel_data_s22", firm_id=1)
